In [ ]:
%pip install tonic-textual
%pip install pyspark
%restart_python

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
  Obtaining dependency information for tonic-textual from https://files.pythonhosted.org/packages/f6/78/bac72b1e7977bb7f58d015b8b7b8ca637711643e7be1ba5f414203dcb00f/tonic_textual-2.1.2-py3-none-any.whl.metadata
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/317.3 MB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.2/317.3 MB 4.8 MB/s eta 0:01:06
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/317.3 MB 53.5 MB/s eta 0:00:06
     ╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/317.3 MB 73.6 MB/s eta 0:00:05
     ━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/317.3 MB 100.3 MB/s eta 0:00:04
     ━╸━━━━━━━━━━━━━━━━━

In [ ]:
from tonic_textual.redact_api import TonicTextual
from pyspark.sql.functions import udf
from pyspark.sql.types import (
    StructType,
    StructField,
    StringType,
    ArrayType,
    IntegerType,
    FloatType,
)

textual = TonicTextual(
    "https://textual.tonic.ai",
    api_key="<API Key>",
)

schema = StructType(
    [
        StructField("original_text", StringType()),
        StructField("redacted_text", StringType()),
        StructField("usage", IntegerType()),
        StructField(
            "de_identify_results",
            ArrayType(
                StructType(
                    [
                        StructField("start", IntegerType()),
                        StructField("end", IntegerType()),
                        StructField("label", StringType()),
                        StructField("text", StringType()),
                        StructField("score", FloatType()),
                    ]
                )
            ),
        ),
    ]
)

sdk_udf = udf(textual.redact, schema)
df = spark.read.table("`textual-demo`.users.tech_demo_people")
df_with_redacted = df.withColumn("Redacted Struct", sdk_udf("Biography"))
df_final = df_with_redacted.select(
    "*",
    df_with_redacted["Redacted Struct"]
    .getItem("redacted_text")
    .alias("Redacted Biography"),
    df_with_redacted["Redacted Struct"]
    .getItem("de_identify_results")
    .alias("DeIdentify Results"),
).drop("Redacted Struct")

display(df_final)

First,Last,Biography,Redacted Biography,DeIdentify Results
Grace,Hopper,Grace Hopper was an American computer scientist and United States Navy rear admiral. She was a pioneer of computer programming who invented one of the first linkers. Hopper is best known for her trailblazing contributions to developing the first compiler for a computer programming language.,[NAME_GIVEN_Ts2H4D] [NAME_FAMILY_FRqr4i5] was an [NRP_GQoUgtj3t] [OCCUPATION_hhowscpU9R864Ip9HxPr0] and [ORGANIZATION_oX9XjewlMli3CloXg4CS1] [OCCUPATION_YQcwzUtBnWPi74]. [GENDER_IDENTIFIER_Ewm1] was a pioneer of computer programming who invented [NUMERIC_VALUE_a5A2] of the [NUMERIC_VALUE_jv4svH] linkers. [NAME_FAMILY_FRqr4i5] is best known for [GENDER_IDENTIFIER_vPC0] trailblazing contributions to developing the [NUMERIC_VALUE_jv4svH] compiler for a computer programming language.,"List(List(0, 5, NAME_GIVEN, Grace, 0.9), List(6, 12, NAME_FAMILY, Hopper, 0.9), List(20, 28, NRP, American, 0.85), List(29, 47, OCCUPATION, computer scientist, 0.9), List(52, 70, ORGANIZATION, United States Navy, 0.9), List(71, 83, OCCUPATION, rear admiral, 0.9), List(85, 88, GENDER_IDENTIFIER, She, null), List(140, 143, NUMERIC_VALUE, one, 0.5), List(151, 156, NUMERIC_VALUE, first, 0.85), List(166, 172, NAME_FAMILY, Hopper, 0.9), List(191, 194, GENDER_IDENTIFIER, her, null), List(240, 245, NUMERIC_VALUE, first, 0.85))"
Alan,Turing,"Alan Turing was a British mathematician, logician, cryptanalyst, and computer scientist. He was instrumental in breaking the German Enigma code during World War II and is considered one of the fathers of theoretical computer science and artificial intelligence.","[NAME_GIVEN_idUl6] [NAME_FAMILY_5J1ABb6] was a [NRP_8Z2xB05I] [OCCUPATION_vQ6Moqb0uqpJwW4], [OCCUPATION_SZhrNn6XD], [OCCUPATION_0G6Mjh5mASJ942], and [OCCUPATION_hhowscpU9R864Ip9HxPr0]. [GENDER_IDENTIFIER_0U0] was instrumental in breaking the [NRP_WOJjBZC] [PRODUCT_opgL0KG] code during [EVENT_u43GrhIjfg4Cz2] and is considered [NUMERIC_VALUE_a5A2] of the fathers of theoretical computer science and artificial intelligence.","List(List(0, 4, NAME_GIVEN, Alan, 0.9), List(5, 11, NAME_FAMILY, Turing, 0.9), List(18, 25, NRP, British, 0.85), List(26, 39, OCCUPATION, mathematician, 0.9), List(41, 49, OCCUPATION, logician, 0.9), List(51, 63, OCCUPATION, cryptanalyst, 0.9), List(69, 87, OCCUPATION, computer scientist, 0.9), List(89, 91, GENDER_IDENTIFIER, He, null), List(125, 131, NRP, German, 0.85), List(132, 138, PRODUCT, Enigma, 0.85), List(151, 163, EVENT, World War II, 0.85), List(182, 185, NUMERIC_VALUE, one, 0.5))"
Margaret,Hamilton,"Margaret Hamilton is an American computer scientist and systems engineer. She led the team that developed the software for the Apollo space program, ensuring the success of the moon landings. Her work laid the foundation for modern software engineering.","[NAME_GIVEN_GQqRfZ2Z7] [NAME_FAMILY_qTAuf01h0] is an [NRP_GQoUgtj3t] [OCCUPATION_hhowscpU9R864Ip9HxPr0] and [OCCUPATION_suZNgWHqkolr7E2jQH]. [GENDER_IDENTIFIER_Ewm1] led the team that developed the software for the [ORGANIZATION_r3jKNeF] space program, ensuring the success of the moon landings. [GENDER_IDENTIFIER_0sf1] work laid the foundation for modern software engineering.","List(List(0, 8, NAME_GIVEN, Margaret, 0.9), List(9, 17, NAME_FAMILY, Hamilton, 0.9), List(24, 32, NRP, American, 0.85), List(33, 51, OCCUPATION, computer scientist, 0.9), List(56, 72, OCCUPATION, systems engineer, 0.9), List(74, 77, GENDER_IDENTIFIER, She, null), List(127, 133, ORGANIZATION, Apollo, 0.85), List(192, 195, GENDER_IDENTIFIER, Her, null))"
Dennis,Ritchie,"Dennis Ritchie was an American computer scientist who created the C programming language and, with long-time colleague Ken Thompson, the Unix operating system. His work has had a lasting impact on the field of computer science and the development of modern software.","[NAME_GIVEN_0PP0swD] [NAME_FAMILY_MTGdCLCT] was an [NRP_GQoUgtj3t] [OCCUPATION_hhowscpU9R864Ip9HxPr0] who created the C program